In [1]:
import os
import shutil
import pandas as pd
pd.set_option('display.max_colwidth', 100)


In [2]:
import faiss
assert faiss.get_num_gpus() > 0

In [3]:

import pyterrier as pt
if not pt.started():
    pt.init()


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
docno_to_text = pd.read_csv('CHS-2021/documents/Webdoc/crawl/txt_over_50.tsv', sep='\t')

In [5]:
def get_text_from_docno(docno):
    print(docno)
    if docno_to_text[docno_to_text['docid'] == docno].empty:
        return ''
    return docno_to_text[docno_to_text['docid'] == docno]['text'].values[0]

In [6]:
def ds_generate(use_chatgpt=True):
    with open('CHS-2021/documents/Webdoc/crawl/txt_over_50.tsv', 'r') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}
    if use_chatgpt:
        for idx, row in chat_gpt_results.iterrows():
            docno = 'chatgpt' + str(row['topic_id'])
            yield {'docno' : docno, 'text' : row['answer']}

use_colbert = True
use_chatgpt = False

if use_chatgpt:
    chat_gpt_results_infile = 'answers/chatgpt-16052023.csv'
    chat_gpt_results = pd.read_csv(chat_gpt_results_infile, header=None)
    chat_gpt_results.columns = ['topic_id', 'query', 'answer']
    chat_gpt_results['topic_id'] = chat_gpt_results['topic_id'].astype(str)
    chat_gpt_results['answer'] = chat_gpt_results['answer'].str.replace('\n', ' ').str.replace(',', ' ').str.replace('"', '')
    chat_gpt_results = chat_gpt_results[chat_gpt_results['answer'].notna()]

if use_colbert:
    index_path = './colbert_index'
    if use_chatgpt:
        index_path += '_chatgpt'
    if os.path.exists(index_path):  
        shutil.rmtree(index_path)
    from pyterrier_colbert.indexing import ColBERTIndexer
    # checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"
    checkpoint="./colbert_model_checkpoint/colbert.dnn"

    indexer = ColBERTIndexer(checkpoint, index_path, "colbertindex", chunksize=3)
    indexref = indexer.index(ds_generate(use_chatgpt))
else:
    index_path = './IterDict_index_over_50'
    if use_chatgpt:
        index_path += '_chatgpt'
    if os.path.exists(index_path):  
        shutil.rmtree(index_path)
    iter_indexer = pt.IterDictIndexer(index_path)
    indexref = iter_indexer.index(ds_generate(use_chatgpt), meta={'docno' : 36, 'text': 2000})

/home/jonas/miniconda3/envs/faiss/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[Jun 22, 13:31:01] [0] 		 #> Local args.bsize = 128
[Jun 22, 13:31:01] [0] 		 #> args.index_root = ./colbert_index
[Jun 22, 13:31:01] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Jun 22, 13:31:04] #> Loading model checkpoint.
[Jun 22, 13:31:04] #> Loading checkpoint ./colbert_model_checkpoint/colbert.dnn
[Jun 22, 13:31:06] #> checkpoint['epoch'] = 0
[Jun 22, 13:31:06] #> checkpoint['batch'] = 44500




[Jun 22, 13:31:06] #> Creating directory ./colbert_index 




[Jun 22, 13:31:06] #> Creating directory ./colbert_index/colbertindex 


[Jun 22, 13:33:46] [0] 		 #> Completed batch #0 (starting at passage #0) 		Passages/min: 2.5k (overall),  2.5k (this encoding),  18111.3M (this saving)
[Jun 22, 13:33:47] [0] 		 [NOTE] Done with local share.
[Jun 22, 13:33:47] [0] 		 #> Joining saver thread.
[Jun 22, 13:33:47] [0] 		 #> Saved batch #0 to ./colbert_index/colbertindex/0.pt 		 Saving Throughput = 613.6k passages per minute.

#> num_embeddings = 1202218
[Jun 22, 13:33:47] #> Starting..
[Jun 22, 13:33:47] #> Processing slice #1 of 1 (range 0..1).
[Jun 22, 13:33:47] #> Will write to ./colbert_index/colbertindex/ivfpq.100.faiss.
[Jun 22, 13:33:47] #> Loading ./colbert_i

In [7]:
if use_colbert:
    from pyterrier_colbert.ranking import ColBERTFactory
    index=(index_path, "colbertindex")
    pytcolbert = ColBERTFactory(checkpoint, *index)
    # rename the ivfpq file
    os.rename(index_path + '/colbertindex/ivfpq.100.faiss', index_path + '/colbertindex/ivfpq.faiss')
    dense_e2e = pytcolbert.end_to_end()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Jun 22, 13:34:10] #> Loading model checkpoint.
[Jun 22, 13:34:10] #> Loading checkpoint ./colbert_model_checkpoint/colbert.dnn
[Jun 22, 13:34:12] #> checkpoint['epoch'] = 0
[Jun 22, 13:34:12] #> checkpoint['batch'] = 44500
[Jun 22, 13:34:12] #> Loading the FAISS index from ./colbert_index/colbertindex/ivfpq.faiss ..
[Jun 22, 13:34:12] #> Building the emb2pid mapping..
[Jun 22, 13:34:12] len(self.emb2pid) = 1202218
Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 1/1 [00:00<00:00,  1.79shard/s]


In [8]:
import xml.etree.ElementTree as ET
import pandas as pd

def load_topics(path, clean_queries=False):
    with open(path) as f:
        root = ET.fromstring(f.read())
    topic_dict = {}
    for topic in root.findall("topic"):
        topic_id = topic.findtext("id")
        topic_query = topic.findtext("query")
        if topic_id and topic_query:
            topic_dict[topic_id] = topic_query.strip()
    topics = pd.DataFrame(topic_dict.items(), columns=["qid", "query"]) 
    if clean_queries:
        topics["query"] = topics["query"].str.lower().replace(r'\W+', ' ', regex=True)
    return topics

In [9]:
if use_colbert:
    clean_queries = False
else:
    clean_queries = True
topics = load_topics("data/topics/topics.txt", clean_queries=True)
qrels = pt.io.read_qrels("data/assessments/qrels.txt") # type: ignore
qcred = pt.io.read_qrels("data/assessments/qcredibility.txt") # type: ignore
qread = pt.io.read_qrels("data/assessments/qreadability.txt") # type: ignore

all_qs = [("qrels", qrels), ("qcred", qcred), ("qread", qread)]

# remove non alphanumeric characters from queries

In [10]:
import pyterrier as pt

def run_experiment(pipeline, simple_name, topics, qrels, eval_metrics=["map", "bpref", "ndcg_cut_10"]):
    experiments = []
    for name, q in qrels:
        # change pipeline name to include the name of the query

        exp = pt.Experiment([pipeline], topics, q, eval_metrics, names=[name + '_' + simple_name])
        experiments.append(exp)
    return pd.concat(experiments, axis=0)

In [11]:
credibility_scores = pd.read_csv('data/all_passages_credibility_scores_bert.tsv', sep='\t')

In [12]:
#get the credibility score for a given docid
def get_credibility_score(docid):
    if credibility_scores[credibility_scores['docid'] == docid].empty:
        return 0
    return credibility_scores[credibility_scores['docid'] == docid]['credibility_score'].values[0]


In [13]:
import textstat
# rank documents with custom function that evaluates readability of the document
def readability_score(text):
    score = textstat.flesch_reading_ease(text)
    print(score)
    return score 

In [14]:
if use_colbert:
    retrieval = dense_e2e
    simple_name = 'colbert_msmarco_over_50'
else:
    dph = pt.BatchRetrieve(indexref, wmodel="DPH", metadata=["docno", "text"])
    # tfidf = pt.BatchRetrieve(indexref, wmodel="TF_IDF", metadata=["docno", "text"])
    bo1 = pt.rewrite.Bo1QueryExpansion(indexref)
    # readability_rerank = pt.apply.doc_score(lambda row: get_credibility_score(row['docid']))
    pipelineQE_dph = dph >> bo1 >> dph # >> readability_rerank
    simple_name = 'QE_dph_over_50'
    retrieval = pipelineQE_dph
results = run_experiment(retrieval,simple_name, topics, all_qs, ["map", "bpref", "ndcg_cut_10"])

In [15]:
append_to_file = ''
if use_chatgpt:
    append_to_file = '_chatgpt'
else:
    append_to_file = '_without_chatgpt'

results.to_csv('data/results/results_' + simple_name + append_to_file + '_clean_queries.csv', index=False)

In [16]:
results

,name,map,bpref,ndcg_cut_10
0,qrels_colbert_msmarco_over_50,0.219678,0.424791,0.483103
0,qcred_colbert_msmarco_over_50,0.306242,0.646405,0.446899
0,qread_colbert_msmarco_over_50,0.248804,0.477628,0.559790


In [17]:
# iterate over topics and save the positions of chatgpt answers
if use_colbert:
    retrieval_model = dense_e2e
else:
    retrieval_model = pipelineQE_dph
out_df = []
for idx, row in topics.iterrows():
    if str(row['qid']) not in chat_gpt_results['topic_id'].values:
        continue
    res = retrieval_model.search(row["query"])
    best_answer_text = get_text_from_docno(res['docno'].tolist()[0])
    docno = 'chatgpt' + str(row['qid'])
    if docno not in res['docno'].values:
        out_df.append({'qid': row['qid'], 'query': row['query'], 'docno': docno, 'position': -1, 'chatgpt_answer': chat_gpt_results[chat_gpt_results['topic_id'] == str(row['qid'])]['answer'].values[0][:2000], 'best_answer': best_answer_text})
        continue
    position = res['docno'].tolist().index(docno)
    out_df.append({'qid': row['qid'], 'query': row['query'], 'docno': docno, 'position': position, 'chatgpt_answer': chat_gpt_results[chat_gpt_results['topic_id'] == str(row['qid'])]['answer'].values[0][:2000], 'best_answer': best_answer_text})
    
out_df = pd.DataFrame(out_df)

NameError: name 'chat_gpt_results' is not defined

In [ ]:
if use_colbert:
    out_file = 'answers/chatgpt-16052023-positions-colbert.csv'
else:
    out_file = 'answers/chatgpt-16052023-positions-IterDict.csv'
out_df.to_csv(out_file, index=False)